In [14]:
import random

def check():
    max = 0
    maxes = 0
    for i in range(1000):
        ch = random.randint(0,100000)
        if ch > max:
            max = ch
            maxes += 1
    return maxes


In [18]:
import numpy as np
print(np.mean([check() for i in range(1000)]))

7.447


In [41]:
import tflearn
import numpy as np
from tflearn.layers.conv import conv_2d, max_pool_2d, upsample_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.recurrent import gru

x = input_data(shape=[None, 28, 28, 1], name='input')  # adapt this if using `channels_first` image data format

x = conv_2d(x, 16, (3, 3), activation='relu')
x = max_pool_2d(x, (2, 2), padding='same')
x = conv_2d(x, 8, (3, 3), activation='relu')
x = max_pool_2d(x, (2, 2), padding='same')
x = conv_2d(x, 8, (3, 3), activation='relu')
encoded = max_pool_2d(x, (2, 2))

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = conv_2d(encoded, 8, (3, 3), activation='relu')
x = upsample_2d (x, (2, 2))
x = conv_2d(x, 8, (3, 3), activation='relu')
x = upsample_2d (x, (2, 2))
x = conv_2d(x, 16, (3, 3), activation='relu')
x = upsample_2d(x, (2, 2))
decoded = conv_2d(x, 1, (3, 3), activation='sigmoid')

autoencoder = regression(decoded, optimizer='momentum',loss='categorical_crossentropy',
                         learning_rate=0.001, name='targets')
model = tflearn.DNN(autoencoder, checkpoint_path='autoencoder',
                        max_checkpoints=1, tensorboard_verbose=0, tensorboard_dir='log')

from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

#model.predict(x_train[0].reshape(-1, 28, 28,1))

model.fit({'input': x_train}, {'targets': x_train}, n_epoch=50, validation_set=({'input': x_test}, {'targets': x_test}),
          snapshot_step=500, show_metric=True, run_id="autoencoder", shuffle=True, )


Exception: Feed dict asks for variable named 'input' but no such variable is known to exist